# Домашнее задание: Visual Question Answering (VQA)

## 1. Введение

### Что такое VQA?

**Visual Question Answering (VQA)** — задача, в которой модель должна ответить на текстовый вопрос об изображении. Например:
- Изображение: фотография кота
- Вопрос: "Какого цвета кот?"
- Ответ: "Рыжий"

Это мультимодальная задача, требующая понимания как визуальной, так и текстовой информации

### Зачем нужны мультимодальные модели?

Традиционные модели работают либо с изображениями, либо с текстом. Мультимодальные модели объединяют оба типа данных:
- **Простой подход:** объединение эмбеддингов из разных моделей (ResNet + T5)
- **Продвинутый подход:** сквозное обучение (CLIP, LLaVA)

## 2. Подготовка окружения

Установим необходимые библиотеки для работы с моделями и интерфейсами.

In [ ]:
!pip install -q torch torchvision transformers open_clip_torch gradio pillow pandas accelerate bitsandbytes

In [ ]:
import torch
import torchvision
from torchvision import transforms, models
from transformers import T5EncoderModel, T5Tokenizer, CLIPProcessor, CLIPModel
from transformers import AutoProcessor, LlavaForConditionalGeneration
import open_clip
import gradio as gr
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torch import nn
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Память: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")


## 3. Подготовка данных

Создадим небольшой датасет для тестирования. Для простоты возьмем несколько изображений из CIFAR-10 и составим вопросы вручную

### Задание 3.1: Загрузите датасет CIFAR-10

**Что нужно сделать:**
- Загрузите тестовую часть CIFAR-10 (используйте `torchvision.datasets.CIFAR10`)
- Выберите 5-7 изображений из разных классов
- Сохраните их в список `sample_images`

In [ ]:
cifar_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True)
sample_images = []
sample_labels = []

cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                 'dog', 'frog', 'horse', 'ship', 'truck']

selected_classes = set()
for img, label in cifar_dataset:
    if label not in selected_classes:
        sample_images.append(img)
        sample_labels.append(label)
        selected_classes.add(label)
    if len(sample_images) == 7:
        break

### Задание 3.2: Создайте DataFrame с вопросами и ответами

**Что нужно сделать:**
- Для каждого изображения придумайте 1-2 вопроса
- Вопросы могут быть о: цвете, типе объекта, количестве объектов, действиях
- Создайте pandas DataFrame с колонками: `image_id`, `question`, `answer`

In [ ]:
qa_data = {
    'image_id': [],
    'question': [],
    'answer': []
}

questions_answers = [
    [(0, 'What object is in the image?', cifar_classes[sample_labels[0]]),
     (0, 'Is this a vehicle?', 'yes' if sample_labels[0] in [0, 1, 8, 9] else 'no')],
    [(1, 'What object is in the image?', cifar_classes[sample_labels[1]]),
     (1, 'Is this an animal?', 'yes' if sample_labels[1] in [2, 3, 4, 5, 6, 7] else 'no')],
    [(2, 'What object is in the image?', cifar_classes[sample_labels[2]]),
     (2, 'Can this fly?', 'yes' if sample_labels[2] in [0, 2] else 'no')],
    [(3, 'What object is in the image?', cifar_classes[sample_labels[3]]),
     (3, 'Is this a living creature?', 'yes' if sample_labels[3] in [2, 3, 4, 5, 6, 7] else 'no')],
    [(4, 'What object is in the image?', cifar_classes[sample_labels[4]]),
     (4, 'Is this used for transportation?', 'yes' if sample_labels[4] in [0, 1, 8, 9] else 'no')],
    [(5, 'What object is in the image?', cifar_classes[sample_labels[5]]),
     (5, 'Does this have legs?', 'yes' if sample_labels[5] in [3, 4, 5, 6, 7] else 'no')],
    [(6, 'What object is in the image?', cifar_classes[sample_labels[6]]),
     (6, 'Is this found in water?', 'yes' if sample_labels[6] in [6, 8] else 'no')]
]

for qa_list in questions_answers:
    for img_id, q, a in qa_list:
        qa_data['image_id'].append(img_id)
        qa_data['question'].append(q)
        qa_data['answer'].append(a)

df = pd.DataFrame(qa_data)
print(f"\n Создан датасет: {len(df)} вопросов для {len(sample_images)} изображений")
print(df.head())

In [ ]:
def visualize_samples(images, df, n_samples=3):
    fig, axes = plt.subplots(1, min(n_samples, len(images)), figsize=(15, 5))
    if n_samples == 1:
        axes = [axes]

    for idx, ax in enumerate(axes):
        if idx < len(images):
            ax.imshow(images[idx])
            ax.axis('off')
            questions = df[df['image_id'] == idx]
            title = f"Image {idx}\n"
            for _, row in questions.iterrows():
                title += f"Q: {row['question'][:30]}...\n"
            ax.set_title(title, fontsize=10)
    plt.tight_layout()
    plt.show()

visualize_samples(sample_images, df, n_samples=3)

## 4. Baseline: ResNet + T5

Создадим простой бейз, который:
1. Извлекает эмбеддинги изображений через предобученный ResNet50
2. Извлекает эмбеддинги вопросов через T5-small
3. Объединяет их и предсказывает ответ через MLP

### Задание 4.1: Извлеките эмбеддинги изображений

**Что нужно сделать:**
- Загрузите предобученный ResNet50
- Удалите последний слой классификации (голову)
- Извлеките эмбеддинги для всех изображений

In [ ]:
class ImageEncoder:
    def __init__(self):
        self.model = models.resnet50(pretrained=True)
        self.model = nn.Sequential(*list(self.model.children())[:-1])
        self.model.eval()
        self.model.to(device)
        self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])
    
    def encode(self, images):
        batch = torch.stack([self.transform(img) for img in images]).to(device)
        with torch.no_grad():
            embeddings = self.model(batch)
        return embeddings.squeeze()

image_encoder = ImageEncoder()
image_embeddings = image_encoder.encode(sample_images)

print(f"Размерность эмбеддингов изображений: {image_embeddings.shape if image_embeddings is not None else 'Fuck'}")

### Задание 4.2: Извлеките эмбеддинги вопросов

**Что нужно сделать:**
- Загрузите T5-small encoder и tokenizer
- Токенизируйте все вопросы
- Получите эмбеддинги (используйте mean pooling по последней скрытой размерности)

In [ ]:
class TextEncoder:
    def __init__(self, model_name='t5-small'):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5EncoderModel.from_pretrained(model_name)
        self.model.eval()
        self.model.to(device)
    
    def encode(self, texts):
        inputs = self.tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = self.model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings

text_encoder = TextEncoder()
question_embeddings = text_encoder.encode(df['question'].tolist())
print(f'Размерность эмбеддингов вопросов: {question_embeddings.shape if question_embeddings is not None else ""}')

### Задание 4.3: Обучите MLP-классификатор

**Что нужно сделать:**
- Объедините эмбеддинги изображений и вопросов (конкатенация)
- Создайте словарь всех уникальных ответов
- Реализуйте простой MLP (2-3 слоя)
- Обучите модель на нескольких эпохах

**Примечание:** Из-за маленького датасета не ожидайте высокую точность. Цель — понять архитектуру.

In [ ]:
class VQAClassifier(nn.Module):
    def __init__(self, image_dim, text_dim, num_classes, hidden_dim=512):
        super().__init__()
        self.fc1 = nn.Linear(image_dim + text_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, image_emb, text_emb):
        x = torch.cat([image_emb, text_emb], dim=1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

answer_vocab = {ans: idx for idx, ans in enumerate(df['answer'].unique())}
idx_to_answer = {idx: ans for ans, idx in answer_vocab.items()}
print(f"\nСловарь ответов ({len(answer_vocab)} классов): {list(answer_vocab.keys())}")

model = VQAClassifier(image_embeddings.shape[1], question_embeddings.shape[1], len(answer_vocab))
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
img_emb_per_question = torch.stack([image_embeddings[img_id] for img_id in df['image_id']]).to(device)
labels = torch.tensor([answer_vocab[ans] for ans in df['answer']]).to(device)

num_epochs = 100
model.train()
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(img_emb_per_question, question_embeddings)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

model.eval()
print('\nОбучение завершено')

### Задание 4.4: Протестируйте baseline

**Что нужно сделать:**
- Выберите 2-3 примера из датасета
- Получите предсказания модели
- Выведите изображение, вопрос, истинный и предсказанный ответ

In [ ]:
def predict_baseline(image_id, question):
    img_emb = image_embeddings[image_id].unsqueeze(0).to(device)
    q_emb = text_encoder.encode([question])
    with torch.no_grad():
        output = model(img_emb, q_emb)
        pred_idx = torch.argmax(output, dim=1).item()
    return idx_to_answer[pred_idx]

for i in range(min(3, len(df))):
    row = df.iloc[i]
    pred = predict_baseline(row['image_id'], row['question'])
    print(f"\nПример {i+1}:")
    print(f"Image ID: {row['image_id']}")
    print(f"Question: {row['question']}")
    print(f"True Answer: {row['answer']}")
    print(f"Predicted: {pred}")

## 5. CLIP Zero-Shot Baseline

CLIP — это мультимодальная модель, обученная связывать изображения и тексты. Мы используем её для zero-shot VQA:
1. Для каждой пары (изображение, вопрос) сформируем набор возможных ответов
2. Составим промпты типа "A photo of {answer}"
3. CLIP выберет наиболее вероятный ответ

### Задание 5.1: Загрузите CLIP

**Что нужно сделать:**
- Загрузите CLIP модель (используйте `openai/clip-vit-base-patch32` через transformers)
- Или используйте `open_clip` библиотеку

In [ ]:
clip_model = CLIPModel.from_pretrained('openai/clip-vit-base-patch32')
clip_processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32')
clip_model.to(device)
clip_model.eval()

### Задание 5.2: Реализуйте zero-shot VQA с CLIP

**Что нужно сделать:**
- Для каждого изображения и вопроса создайте список возможных ответов (используйте answer_vocab)
- Сформируйте промпты: \"Question: {question}. Answer: {answer}\"
- Используйте CLIP для выбора наиболее подходящего ответа

In [ ]:
def predict_clip(image, question, candidate_answers):
    prompts = [f"Question: {question}. Answer: {answer}" for answer in candidate_answers]
    
    inputs = clip_processor(text=prompts, images=image, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = clip_model(**inputs)
    
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)
    best_idx = probs.argmax().item()
    
    return candidate_answers[best_idx]

candidate_answers = list(answer_vocab.keys())
for i in range(min(3, len(df))):
    row = df.iloc[i]
    pred = predict_clip(sample_images[row['image_id']], row['question'], candidate_answers)
    print(f"\nПример {i+1}:")
    print(f"Question: {row['question']}")
    print(f"True Answer: {row['answer']}")
    print(f"CLIP Predicted: {pred}")

## 6. LLaVA Inference

LLaVA (Large Language and Vision Assistant) — это большая мультимодальная модель, которая может генерировать текстовые ответы на вопросы об изображениях.

**Внимание:** LLaVA-1.5-7B требует ~14GB GPU памяти. Если в Colab недостаточно памяти, используйте квантизацию (8-bit) или напишите мне про датасферу.

### Задание 6.1: Загрузите LLaVA

**Что нужно сделать:**
- Загрузите модель `llava-hf/llava-1.5-7b-hf`
- При необходимости используйте квантизацию для экономии памяти

In [ ]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16
)

llava_model = LlavaForConditionalGeneration.from_pretrained(
    "llava-hf/llava-1.5-7b-hf",
    quantization_config=quantization_config,
    device_map="auto"
)
llava_processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")

### Задание 6.2: Генерация ответов с LLaVA

**Что нужно сделать:**
- Реализуйте функцию для генерации ответов
- Используйте формат промпта: "USER: <image>
Question: {question}
ASSISTANT:"
- Протестируйте на 2-3 примерах

In [ ]:
def predict_llava(image, question):
    prompt = f"USER: <image>\nQuestion: {question}\nASSISTANT:"
    
    inputs = llava_processor(text=prompt, images=image, return_tensors="pt")
    inputs = {k: v.to(llava_model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        output = llava_model.generate(**inputs, max_new_tokens=50)
    
    answer = llava_processor.decode(output[0], skip_special_tokens=True)
    answer = answer.split("ASSISTANT:")[-1].strip()
    
    return answer

for i in range(min(3, len(df))):
    row = df.iloc[i]
    pred = predict_llava(sample_images[row['image_id']], row['question'])
    print(f"\nПример {i+1}:")
    print(f"Question: {row['question']}")
    print(f"True Answer: {row['answer']}")
    print(f"LLaVA Predicted: {pred}")

## 7. Сравнение результатов

Теперь сравним все три подхода на одних и тех же примерах.

### Задание 7.1: Соберите результаты всех моделей

Что нужно сделать:
- Для каждого примера из датасета получите предсказания от всех трёх моделей
- Создайте сравнительную таблицу
- Проанализируйте, где какая модель работает лучше

In [ ]:
results = {
    'Image ID': [],
    'Question': [],
    'True Answer': [],
    'ResNet+T5': [],
    'CLIP': [],
    'LLaVA': []
}

for i, row in df.iterrows():
    image = sample_images[row['image_id']]
    question = row['question']
    
    results['Image ID'].append(row['image_id'])
    results['Question'].append(question)
    results['True Answer'].append(row['answer'])
    results['ResNet+T5'].append(predict_baseline(row['image_id'], question))
    results['CLIP'].append(predict_clip(image, question, candidate_answers))
    results['LLaVA'].append(predict_llava(image, question))

results_df = pd.DataFrame(results)
results_df

### Задание 7.2: Проанализируйте результаты

Что нужно сделать:
- Посчитайте accuracy для каждой модели
- Опишите сильные и слабые стороны каждого подхода
- Приведите примеры, где модели ошибаются или дают разные ответы

In [ ]:
def calculate_accuracy(predictions, true_answers):
    correct = sum([1 for pred, true in zip(predictions, true_answers) if pred.lower() == true.lower()])
    return correct / len(true_answers)

baseline_acc = calculate_accuracy(results_df['ResNet+T5'], results_df['True Answer'])
clip_acc = calculate_accuracy(results_df['CLIP'], results_df['True Answer'])
llava_acc = calculate_accuracy(results_df['LLaVA'], results_df['True Answer'])

print("\nТочность моделей:")
print(f"ResNet+T5: {baseline_acc:.2%}")
print(f"CLIP: {clip_acc:.2%}")
print(f"LLaVA: {llava_acc:.2%}")

In [ ]:
import matplotlib.pyplot as plt
models = ['ResNet+T5', 'CLIP', 'LLaVA']
accuracies = [baseline_acc, clip_acc, llava_acc]
plt.bar(models, accuracies)
plt.ylabel('Accuracy')
plt.title('Model Comparison')
plt.ylim([0, 1])
plt.show()

### Выводы (заполните после экспериментов):

Baseline (ResNet + T5):
- Сильные стороны: [Ваш ответ]
- Слабые стороны: [Ваш ответ]

CLIP:
- Сильные стороны: [Ваш ответ]
- Слабые стороны: [Ваш ответ]

LLaVA:
- Сильные стороны: [Ваш ответ]
- Слабые стороны: [Ваш ответ]

Общие наблюдения:
[Ваши выводы о том, какие модели лучше справляются с разными типами вопросов]

Мнения о домашке: [Ваш ответ]